In [4]:
import numpy as np # mathematical operations and algebra
import pandas as pd # data processing, CSV file I/O
import seaborn as sns # Fancier visualizations
sns.set(color_codes=True)
import matplotlib.pyplot as plt # visualization library
import os
import scipy.stats as stats 

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
print("welcome guys")